# GPT 动作库：BigQuery

## 简介

本页面为开发人员构建特定应用程序的GPT动作提供了指导和指南。在继续之前，请确保首先熟悉以下信息：- [GPT动作简介](https://platform.openai.com/docs/actions)- [GPT动作库简介](https://platform.openai.com/docs/actions-library)- [从头开始构建GPT动作的示例](https://platform.openai.com/docs/getting-started)

这个特定的GPT操作提供了如何连接到**Google BigQuery**的概述，Google Cloud的分析数据仓库。该操作接收用户的问题，扫描相关表以收集数据模式，然后编写一个SQL查询来回答用户的问题。注意：这些说明返回一个可用的SQL语句，而不是结果本身。目前需要中间件来返回一个CSV文件 - 我们将很快发布关于这方面示例的说明。

## 应用程序信息

### 应用密钥链接

在开始之前，请查看应用程序的以下链接：- 应用程序网站：https://cloud.google.com/bigquery- 应用程序API文档：https://cloud.google.com/bigquery/docs/reference/rest

### 应用先决条件

在开始之前，请确保在应用程序环境中完成以下步骤：- 设置一个GCP项目- 在该GCP项目中设置一个BQ数据集- 确保通过ChatGPT认证进入BigQuery的用户可以访问该BQ数据集

## ChatGPT 步骤

### 自定义GPT指令

创建自定义GPT后，请将下面的文本复制到“说明”面板中。有问题吗？查看[入门示例](https://platform.openai.com/docs/getting-started)以了解更详细的操作步骤。

In [ ]:
**Context**: You are an expert at writing BigQuery SQL queries. A user is going to ask you a question. **Instructions**:1. No matter the user's question, start by running `runQuery` operation using this query: "SELECT column_name, table_name, data_type, description FROM `{project}.{dataset}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`" -- Assume project = "<insert your default project here>", dataset = "<insert your default dataset here>", unless the user provides different values -- Remember to include useLegacySql:false in the json output2. Convert the user's question into a SQL statement that leverages the step above and run the `runQuery` operation on that SQL statement to confirm the query works. Add a limit of 100 rows3. Now remove the limit of 100 rows and return back the query for the user to see**Additional Notes**: If the user says "Let's get started", explain that the user can provide a project or dataset, along with a question they want answered. If the user has no ideas, suggest that we have a sample flights dataset they can query - ask if they want you to query that

### OpenAPI模式

创建了自定义GPT后，在“操作”面板中复制下面的文本。有问题吗？查看[入门示例](https://platform.openai.com/docs/getting-started)以了解这一步骤的更详细操作。

In [ ]:
openapi: 3.1.0info:  title: BigQuery API  description: API for querying a BigQuery table.  version: 1.0.0servers:  - url: https://bigquery.googleapis.com/bigquery/v2    description: Google BigQuery API serverpaths:  /projects/{projectId}/queries:    post:      operationId: runQuery      summary: Executes a query on a specified BigQuery table.      description: Submits a query to BigQuery and returns the results.      parameters:        - name: projectId          in: path          required: true          description: The ID of the Google Cloud project.          schema:            type: string      requestBody:        required: true        content:          application/json:            schema:              type: object              properties:                query:                  type: string                  description: The SQL query string.                useLegacySql:                  type: boolean                  description: Whether to use legacy SQL.                  default: false      responses:        '200':          description: Successful query execution.          content:            application/json:              schema:                type: object                properties:                  kind:                    type: string                    example: "bigquery#queryResponse"                  schema:                    type: object                    description: The schema of the results.                  jobReference:                    type: object                    properties:                      projectId:                        type: string                      jobId:                        type: string                  rows:                    type: array                    items:                      type: object                      properties:                        f:                          type: array                          items:                            type: object                            properties:                              v:                                type: string                  totalRows:                    type: string                    description: Total number of rows in the query result.                  pageToken:                    type: string                    description: Token for pagination of query results.        '400':          description: Bad request. The request was invalid.        '401':          description: Unauthorized. Authentication is required.        '403':          description: Forbidden. The request is not allowed.        '404':          description: Not found. The specified resource was not found.        '500':          description: Internal server error. An error occurred while processing the request.

## 认证说明

以下是设置与这个第三方应用程序进行身份验证的说明。有问题吗？请查看[入门示例](https://platform.openai.com/docs/getting-started)，以了解更详细的操作步骤。

### 预操作步骤

在设置ChatGPT中的身份验证之前，请在应用程序中执行以下步骤。

- 进入Google Cloud控制台- 导航至API和服务 > 凭据- 创建新的OAuth凭据（或使用现有凭据）- 找到您的OAuth客户端ID和客户端密钥，并安全地存储这两个值（参见下面的截图）

![gptactions_BigQuery_auth.png](../../../images/gptactions_BigQuery_auth.png)

### 在ChatGPT

在ChatGPT中，点击“身份验证”，选择**“OAuth”**。输入下面的信息。

- **客户端ID**：使用上述步骤中的客户端ID- **客户端密钥**：使用上述步骤中的客户端密钥- **授权URL**：https://accounts.google.com/o/oauth2/auth- **令牌URL**：https://oauth2.googleapis.com/token- **范围**：https://www.googleapis.com/auth/bigquery- **令牌**：默认（POST）

### 后续步骤

在ChatGPT中设置了身份验证之后，请按照以下步骤在应用程序中完成操作。

- 从GPT Action中复制回调URL- 在“授权重定向URI”（参见上面的截图）中添加您的回调URL

### 常见问题与故障排除

- *回调URL错误：* 如果在ChatGPT中遇到回调URL错误，请仔细查看上面的截图。您需要直接将回调URL添加到GCP中，以便操作能够正确进行身份验证。- *模式调用了错误的项目或数据集：* 如果ChatGPT调用了错误的项目或数据集，请考虑更新您的说明，使其更明确，要求用户在运行查询之前提供准确的项目/数据集细节。

*您希望我们优先考虑哪些集成？我们的集成中是否存在错误？请在我们的github上提交PR或问题，我们会进行查看。*